In [173]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Step-1: Loading of Data

In [174]:
data = pd.read_csv('Assignment_Dataset.csv')
data.head()

,Date,Author,Verified,Helpful,Title,Body,Rating,Images,Videos,URL,Variation,Style
0,"May 12, 2019",Renee,yes,yes,"Great, quality phone for the price!",I've only had the a20 for a week but I absolut...,5.0,https://images-na.ssl-images-amazon.com/images...,-,https://www.amazon.com/gp/customer-reviews/R26...,B07QV2JKGQ,Color: Deep Blue | Style: 32 GB
1,"June 17, 2019",Tech nerd,yes,yes,Best budget phone 2019,This is pretty much a budget device. That bei...,5.0,-,-,https://www.amazon.com/gp/customer-reviews/R2I...,B07R2F3G4Q,"Color: Red | Style: International version, no ..."
2,"June 5, 2019",Brittney,yes,yes,"Good graphics, fast charging, great bezzel",I really like this phone the graphics is gorge...,5.0,https://images-na.ssl-images-amazon.com/images...,-,https://www.amazon.com/gp/customer-reviews/R22...,B07R2F3G4Q,"Color: Red | Style: International version, no ..."
3,"June 26, 2019",wkags,yes,yes,Great Phone,This is the 2nd phone I've purchased in the sa...,5.0,-,-,https://www.amazon.com/gp/customer-reviews/R2R...,B07QV2JKGQ,Color: Deep Blue | Style: 32 GB
4,"May 16, 2019",joseph thompson,yes,yes,Beware,The Wi-Fi don't work,1.0,-,-,https://www.amazon.com/gp/customer-reviews/R10...,B07R2F3G4Q,"Color: Red | Style: International version, no ..."


In [175]:
print(data.shape)
print(data.columns)

(100, 12)
Index(['Date', 'Author', 'Verified', 'Helpful', 'Title', 'Body', 'Rating',
       'Images', 'Videos', 'URL', 'Variation', 'Style'],
      dtype='object')


# Step-2: Exatracting Specific columns

In [176]:
newData = data[['Body', 'Rating']]
newData.isnull()

,Body,Rating
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


In [177]:
newData = newData.dropna(axis=0, how='any')  # dropping of NaN values
newData

,Body,Rating
0,I've only had the a20 for a week but I absolut...,5.0
1,This is pretty much a budget device. That bei...,5.0
2,I really like this phone the graphics is gorge...,5.0
3,This is the 2nd phone I've purchased in the sa...,5.0
4,The Wi-Fi don't work,1.0
5,"I wanted to like the A20, I really did but th...",2.0
6,This phone has a weird glitch.....when I go to...,3.0
7,When i put in the microSD card that it came wi...,4.0
8,⛔👎📵👎😑👎No NFC???? Are you kidding me? How cheap...,1.0
9,Phone was defective..,1.0


In [178]:
y=newData['Rating']
y


0     5.0
1     5.0
2     5.0
3     5.0
4     1.0
5     2.0
6     3.0
7     4.0
8     1.0
9     1.0
10    5.0
11    5.0
12    5.0
13    5.0
14    5.0
15    5.0
16    4.0
17    5.0
18    5.0
19    3.0
20    5.0
21    5.0
22    1.0
23    1.0
24    5.0
25    1.0
26    5.0
27    5.0
28    5.0
29    5.0
     ... 
69    5.0
70    5.0
71    3.0
72    5.0
73    5.0
74    5.0
75    5.0
76    5.0
77    5.0
78    5.0
79    3.0
80    5.0
81    5.0
82    3.0
83    5.0
84    4.0
85    2.0
86    5.0
87    4.0
88    2.0
89    2.0
90    5.0
91    1.0
92    5.0
93    5.0
94    5.0
95    5.0
96    5.0
97    4.0
98    5.0
Name: Rating, Length: 99, dtype: float64

In [180]:
from nltk.stem import WordNetLemmatizer
lemitizer= WordNetLemmatizer()

In [181]:
def cleanup(a):
    a=re.sub('[^A-Za-z]+',' ', a)
    from nltk.stem import WordNetLemmatizer
    lemitizer= WordNetLemmatizer()
    text_lower=a.lower()
    filtered_sentence= lemitizer.lemmatize(text_lower)
    return(filtered_sentence)


In [182]:
filtered_sent=[]
for i in newData['Body']:
    filtered_sent.append(cleanup(i))
    print (filtered_sent)

['i ve only had the a for a week but i absolutely love it great samsung phone for of course it does all the things you need a phone to do make calls text web surf but the display is great and the bezels are so thin i also like that since the phone is unlocked that i was easily able to switch out my sim card from my old phone and put it in this one and it worked immediately btw i have tracfone service plus samsung has so many neat features to their phones like samsung themes also the price is a definite selling point definitely pleased with the purchase ']
['i ve only had the a for a week but i absolutely love it great samsung phone for of course it does all the things you need a phone to do make calls text web surf but the display is great and the bezels are so thin i also like that since the phone is unlocked that i was easily able to switch out my sim card from my old phone and put it in this one and it worked immediately btw i have tracfone service plus samsung has so many neat feat

In [183]:
def word_token(text):
    word_tokens_list=text.split()
    return word_tokens_list

In [184]:
word_tokens=[]
for i in filtered_sent:
    word_tokens.append(word_token(i))
print (word_tokens)
len(word_tokens)

[['i', 've', 'only', 'had', 'the', 'a', 'for', 'a', 'week', 'but', 'i', 'absolutely', 'love', 'it', 'great', 'samsung', 'phone', 'for', 'of', 'course', 'it', 'does', 'all', 'the', 'things', 'you', 'need', 'a', 'phone', 'to', 'do', 'make', 'calls', 'text', 'web', 'surf', 'but', 'the', 'display', 'is', 'great', 'and', 'the', 'bezels', 'are', 'so', 'thin', 'i', 'also', 'like', 'that', 'since', 'the', 'phone', 'is', 'unlocked', 'that', 'i', 'was', 'easily', 'able', 'to', 'switch', 'out', 'my', 'sim', 'card', 'from', 'my', 'old', 'phone', 'and', 'put', 'it', 'in', 'this', 'one', 'and', 'it', 'worked', 'immediately', 'btw', 'i', 'have', 'tracfone', 'service', 'plus', 'samsung', 'has', 'so', 'many', 'neat', 'features', 'to', 'their', 'phones', 'like', 'samsung', 'themes', 'also', 'the', 'price', 'is', 'a', 'definite', 'selling', 'point', 'definitely', 'pleased', 'with', 'the', 'purchase'], ['this', 'is', 'pretty', 'much', 'a', 'budget', 'device', 'that', 'being', 'said', 'it', 'is', 'a', 'del

99

# Through nltk

In [185]:
from nltk.tokenize import word_tokenize
def word_tokens1(text):
    word_tokens=nltk.word_tokenize(text)
    return word_tokens


In [187]:
word_tokens2=[]
for sen in filtered_sent:
    word_tokens2.append(word_tokens1(sen)) 
print (word_tokens2)

[['i', 've', 'only', 'had', 'the', 'a', 'for', 'a', 'week', 'but', 'i', 'absolutely', 'love', 'it', 'great', 'samsung', 'phone', 'for', 'of', 'course', 'it', 'does', 'all', 'the', 'things', 'you', 'need', 'a', 'phone', 'to', 'do', 'make', 'calls', 'text', 'web', 'surf', 'but', 'the', 'display', 'is', 'great', 'and', 'the', 'bezels', 'are', 'so', 'thin', 'i', 'also', 'like', 'that', 'since', 'the', 'phone', 'is', 'unlocked', 'that', 'i', 'was', 'easily', 'able', 'to', 'switch', 'out', 'my', 'sim', 'card', 'from', 'my', 'old', 'phone', 'and', 'put', 'it', 'in', 'this', 'one', 'and', 'it', 'worked', 'immediately', 'btw', 'i', 'have', 'tracfone', 'service', 'plus', 'samsung', 'has', 'so', 'many', 'neat', 'features', 'to', 'their', 'phones', 'like', 'samsung', 'themes', 'also', 'the', 'price', 'is', 'a', 'definite', 'selling', 'point', 'definitely', 'pleased', 'with', 'the', 'purchase'], ['this', 'is', 'pretty', 'much', 'a', 'budget', 'device', 'that', 'being', 'said', 'it', 'is', 'a', 'del

# Bow

In [188]:
from nltk.corpus import stopwords
stopwords1=stopwords.words('english')
cv= CountVectorizer(stop_words=stopwords1) 
print (cv)
cv.fit(filtered_sent) #training algorithm
print (cv.vocabulary_)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',... 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
{'week': 1203, 'absolutely': 2, 'love': 611, 'great': 468, 'samsung': 936, 'phone': 785, 'course': 239, 'things': 1109, 'need': 686, 'make': 625, 'calls': 142, 'text': 1100, 'web': 1201, 'surf': 1074, 'display': 289, 'bezels': 93

In [189]:
bow_list= cv.fit_transform(filtered_sent)
print(bow_list)

  (0, 851)	1
  (0, 808)	1
  (0, 272)	1
  (0, 813)	1
  (0, 963)	1
  (0, 271)	1
  (0, 832)	1
  (0, 1106)	1
  (0, 786)	1
  (0, 384)	1
  (0, 684)	1
  (0, 631)	1
  (0, 811)	1
  (0, 967)	1
  (0, 1136)	1
  (0, 126)	1
  (0, 516)	1
  (0, 1225)	1
  (0, 728)	1
  (0, 856)	1
  (0, 724)	1
  (0, 151)	1
  (0, 988)	1
  (0, 1078)	1
  (0, 1)	1
  :	:
  (97, 29)	1
  (97, 785)	2
  (97, 468)	2
  (98, 190)	1
  (98, 1099)	1
  (98, 478)	1
  (98, 81)	1
  (98, 433)	1
  (98, 343)	1
  (98, 338)	2
  (98, 1194)	1
  (98, 957)	1
  (98, 1050)	1
  (98, 462)	1
  (98, 692)	1
  (98, 547)	1
  (98, 183)	1
  (98, 388)	1
  (98, 1226)	1
  (98, 128)	1
  (98, 384)	1
  (98, 591)	1
  (98, 142)	1
  (98, 785)	3
  (98, 468)	1


In [190]:
len(cv.vocabulary_)
bow_list.shape

(99, 1243)

In [191]:
bow_list1=bow_list.toarray()
bow_list1

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [192]:
for i in bow_list1[0:99]:
    print (i)

[0 1 1 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 1 ... 0 1 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 1 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 1 ... 0 0 0]
[0 0 0 ... 1 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 1]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ...

In [229]:
print (bow_list1.max())
print (bow_list1.min())


14
0


In [227]:
print(bow_list1[72])
print(bow_list[72:73])

[0 0 0 ... 0 0 0]
  (0, 944)	1
  (0, 446)	1
  (0, 70)	1
  (0, 343)	1
  (0, 543)	1
  (0, 137)	1
  (0, 1226)	1
  (0, 785)	2
  (0, 611)	1


# n_grams (1, 3)

In [195]:
from nltk.corpus import stopwords
stopwords1=stopwords.words('english')
cv1= CountVectorizer(stop_words=stopwords1, ngram_range= (1,3)) 
print (cv1)
cv1.fit(filtered_sent) #training algorithm
print (cv1.vocabulary_)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',... 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
{'week': 7478, 'absolutely': 14, 'love': 3714, 'great': 2802, 'samsung': 5925, 'phone': 4697, 'course': 1439, 'things': 6901, 'need': 4118, 'make': 3801, 'calls': 863, 'text': 6845, 'web': 7464, 'surf': 6723, 'display': 1706, 'be

In [196]:
bow_list2= cv.fit_transform(filtered_sent)
print(bow_list2)

  (0, 851)	1
  (0, 808)	1
  (0, 272)	1
  (0, 813)	1
  (0, 963)	1
  (0, 271)	1
  (0, 832)	1
  (0, 1106)	1
  (0, 786)	1
  (0, 384)	1
  (0, 684)	1
  (0, 631)	1
  (0, 811)	1
  (0, 967)	1
  (0, 1136)	1
  (0, 126)	1
  (0, 516)	1
  (0, 1225)	1
  (0, 728)	1
  (0, 856)	1
  (0, 724)	1
  (0, 151)	1
  (0, 988)	1
  (0, 1078)	1
  (0, 1)	1
  :	:
  (97, 29)	1
  (97, 785)	2
  (97, 468)	2
  (98, 190)	1
  (98, 1099)	1
  (98, 478)	1
  (98, 81)	1
  (98, 433)	1
  (98, 343)	1
  (98, 338)	2
  (98, 1194)	1
  (98, 957)	1
  (98, 1050)	1
  (98, 462)	1
  (98, 692)	1
  (98, 547)	1
  (98, 183)	1
  (98, 388)	1
  (98, 1226)	1
  (98, 128)	1
  (98, 384)	1
  (98, 591)	1
  (98, 142)	1
  (98, 785)	3
  (98, 468)	1


In [197]:
bow_list2.shape

(99, 1243)

In [198]:
print (len(cv1.vocabulary_))
bow_list2=bow_list2.toarray()
print (bow_list1.max())
print (bow_list1.min())

7835
14
0


In [199]:
import sklearn
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(bow_list1,y,test_size=0.2,random_state=10)

In [200]:
print (x_test)
print (y_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
19    3.0
14    5.0
43    5.0
37    1.0
66    5.0
3     5.0
80    5.0
41    4.0
38    1.0
68    5.0
2     5.0
1     5.0
60    5.0
53    5.0
88    2.0
74    5.0
91    1.0
26    5.0
59    5.0
46    5.0
Name: Rating, dtype: float64


# Logistic Regression

In [201]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
log_reg=LogisticRegression().fit(x_train,y_train)
y_predict=log_reg.predict(x_test)
print('accuracy score:',accuracy_score(y_predict,y_test))

accuracy score: 0.75


# Multinomial NB

In [204]:
#Multinomial Model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
model = MultinomialNB()
model.fit(x_train, y_train)
predicted = model.predict(x_test)
print(classification_report(y_test, predicted))
accuracy_score(y_test, predicted)

             precision    recall  f1-score   support

        1.0       1.00      0.33      0.50         3
        2.0       0.00      0.00      0.00         1
        3.0       0.00      0.00      0.00         1
        4.0       0.00      0.00      0.00         1
        5.0       0.73      0.79      0.76        14

avg / total       0.66      0.60      0.61        20



0.6

[0 0 0 ... 0 0 0]
  (0, 944)	1
  (0, 446)	1
  (0, 70)	1
  (0, 343)	1
  (0, 543)	1
  (0, 137)	1
  (0, 1226)	1
  (0, 785)	2
  (0, 611)	1


In [225]:
a = np.array(bow_list2[72]) # list nr 
print (a)


[0 0 0 ... 0 0 0]


# Step-3: Noise Removal (Removal of punctuations)

In [203]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [ ]:
newData['Body'] = newData['Body'].apply(remove_punctuation)
newData['Body']
for i in newData['Body']:
    print (i)

# Step-4: Removal of Stopwords

In [ ]:
stop_words = stopwords.words('english')
np.array(stop_words)

In [ ]:

def stopwords(text):
    list=[]
    for w in text:
        if w not in stop_words:
            list.append(w)
    return list
            
            

In [ ]:
word_token_stop=[]
for i in word_tokens:
    word_token_stop.append(word_tokens(i))
print (word_token_stop)

In [ ]:
def stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop_words]
    return " ".join(text)

In [ ]:
newData = newData['Body'].apply(stopwords)
newData['Body']

In [ ]:
newData['Body'] = newData['Body'].apply(stopwords)
newData['Body']

# Step-5 : Lemmatization

In [ ]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [ ]:
#newData['Body'] = newData['Body'].apply(lambda y : word_lemmatizer(y))
#newData['Body']

# Step-6 : Sentence tokenizing

In [ ]:
def sentence_tokenizer(text):
    sent_tokens = nltk.sent_tokenize(text)
    return sent_tokens

In [ ]:
sent_tokens = newData['Body'].apply(lambda y : sentence_tokenizer(y))
newData['Body'] = sent_tokens
newData['Body']

# Step-7 : Word tokenizing

In [ ]:
def word_tokens(text):
    token = []
    for item in text:
        print("Item:", item)
        str1 = " ".join(item)
        print("str1:",str1)
        token.append(nltk.word_tokenize(str1))
    return token
    #word_token = [nltk.word_tokenize(word) for word in text]
    #return word_token

In [ ]:
print(newData['Body'])
newData['Body'] = newData['Body'].apply(lambda x : word_tokens(x))
newData['Body']

# Step-8 : Stemming

In [ ]:
sent_tokens

In [ ]:
stemmer = PorterStemmer()

def word_stemmer(text):
    stemming_text = " ".join([stemmer.stem(i) for i in text])
    return stemming_text

In [ ]:
newData['Body'] = newData['Body'].apply(lambda y : word_stemmer(y))
newData['Body']

# Step-9 : Bag of Words (BoW)

In [ ]:
count_vector = CountVectorizer(stop_words = stop_words)
count_vector.fit(word_tokens)
count_vector.vocabulary_

In [ ]:
def count_vector_fit(text):
    vector = []
    for item in text:
        sentence = ','.join(str(i) for i in item)
        #print(sentence)
        count_vector.fit([sentence])
    return vector

#count_vector.fit(sent_tokens)
#count_vector_transform = count_vector.fit_transform(sent_tokens)
#print(count_vector_transform)

In [ ]:
count_vector_fit(sent_tokens)

In [ ]:
def count_vector_transform_fit(text):
    vector = []
    for item in text:
        sentence = ','.join(str(i) for i in item)
        count_vector.fit_transform([sentence])
    return vector

In [ ]:
cvt = count_vector_transform_fit(sent_tokens)
cvt

In [ ]:
count_vector_transform.toarray()

# Step-10 : ngrams 

In [ ]:
count_vectorizer = CountVectorizer(ngram_range = (1, 3))
bd = newData['Body']
cv_ngram = count_vectorizer.fit_transform(newData['Body'])
cv_ngram

# Step-11 : Tfidf

In [ ]:
tfidf_vector = TfidfVectorizer(ngram_range = (1,2))
tfidf_result = tfidf_vector.fit_transform(newData['Body'])
tfidf_result